# Effects of different protein expression levels

In [ ]:
using RetroSignalModel
import RetroSignalModel as rs
using ModelingToolkit
using Plots
using DifferentialEquations

In [ ]:
# Make positive sine wave
function make_pos_sine(; amplitude=1, frequency=1, phase=0)
    return t -> 0.5 * amplitude * (sinpi(2 * frequency * t + phase) + 1)
end

# Make square wave
function make_square_wave(; hi=1, lo=0, duty=(0.25, 0.75))
    return t -> ifelse(duty[1] <= t <= duty[2], hi, lo)
end

In [ ]:
plot([make_pos_sine(), make_square_wave()], 0, 1, label=["Positive Sine" "Square wave"])

In [ ]:
params = load_parameters("solution_rtgM4.csv")[1]

In [ ]:
tend=500.0
edges = (125.0, 375.0)

In [ ]:
@variables t
signal(t) = edges[begin] <= t <= edges[end] ? 1.0 : 0.0
@register_symbolic signal(t)

@named sys = RtgMTK(signal)

In [ ]:
u0 = rs.resting_u0(sys)
prob = ODEProblem(sys, u0, tend, params)

In [ ]:
sol = solve(prob, tstops=edges)

In [ ]:
plot(sol, vars=(rs.s), xlims=())

In [ ]:
plot(sol, vars=[rs.Rtg1_n])

## Simulations

The product of the following
- Waveforms: Square (125-375 sec duty cycle)
- Parameter to change: 5 Protein levels
- Protein amount: `[0.001,0.01,0.1,0.2,0.5,1.,2.,4. ,8., 16.,50.,100.]` x of original levels.

End point: ratio of nucleus to cytosol concentrations of Rtg1 and Rtg3